In [ ]:
!nvidia-smi

Preparação do ambiente

In [ ]:
# Instalar Ultralytics para a comparação com YOLO
!pip install ultralytics -q

# Importar as bibliotecas necessárias
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

# Importações do PyTorch
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Configurar o dispositivo para GPU se disponível
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Usando dispositivo: {device}')

Montar Drive e Copiar Dataset Original

In [ ]:
# Célula 2: Montar Drive e Copiar Dataset Original

from google.colab import drive
import os

print("Conectando ao Google Drive...")
drive.mount('/content/drive')

# Caminhos de ORIGEM (Google Drive)
drive_project_path = '/content/drive/MyDrive/Projeto_IA_KITTI_Clean'
source_image_dir = os.path.join(drive_project_path, 'training', 'image_2')
source_label_dir = os.path.join(drive_project_path, 'training', 'label_2')

# Caminhos de DESTINO para a cópia local dos dados ORIGINAIS
local_original_image_dir = '/content/training_original/image_2'
local_original_label_dir = '/content/training_original/label_2'

print("\nCopiando dataset original do Drive para o ambiente local...")
!rm -rf /content/training_original
!mkdir -p {local_original_image_dir}
!mkdir -p {local_original_label_dir}
!cp {source_image_dir}/* {local_original_image_dir}
!cp {source_label_dir}/* {local_original_label_dir}

print("Cópia dos dados originais concluída!")

Pré-processar Dados

In [ ]:
# Célula 3: Pré-processar Dados

from tqdm.notebook import tqdm
import cv2

# Caminhos de origem (a cópia local que acabou de ser feita)
source_image_dir = '/content/training_original/image_2'
source_label_dir = '/content/training_original/label_2'

# Caminhos de destino para o novo dataset PRÉ-PROCESSADO
processed_image_dir = '/content/kitti_processed/image_2'
processed_label_dir = '/content/kitti_processed/label_2'

TARGET_SIZE = (600, 600)
os.makedirs(processed_image_dir, exist_ok=True)
os.makedirs(processed_label_dir, exist_ok=True)

print(f"Iniciando pré-processamento. Imagens serão salvas em: {processed_image_dir}")
image_files = os.listdir(source_image_dir)

for img_name in tqdm(image_files, desc="Pré-processando imagens e anotações"):
    img_id = img_name.split('.')[0]
    img_path_orig = os.path.join(source_image_dir, img_name)
    label_path_orig = os.path.join(source_label_dir, f"{img_id}.txt")

    image = cv2.imread(img_path_orig)
    if image is None: continue # Pula imagens corrompidas
    original_h, original_w, _ = image.shape
    resized_image = cv2.resize(image, TARGET_SIZE)
    cv2.imwrite(os.path.join(processed_image_dir, img_name), resized_image)

    new_labels = []
    with open(label_path_orig, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_name = parts[0]
            other_parts = parts[1:4] + parts[8:]
            xmin, ymin, xmax, ymax = map(float, parts[4:8])
            xmin_scaled = xmin * (TARGET_SIZE[0] / original_w); xmax_scaled = xmax * (TARGET_SIZE[0] / original_w)
            ymin_scaled = ymin * (TARGET_SIZE[1] / original_h); ymax_scaled = ymax * (TARGET_SIZE[1] / original_h)
            new_line = f"{class_name} {' '.join(other_parts[:3])} {xmin_scaled:.2f} {ymin_scaled:.2f} {xmax_scaled:.2f} {ymax_scaled:.2f} {' '.join(other_parts[3:])}"
            new_labels.append(new_line)
    with open(os.path.join(processed_label_dir, f"{img_id}.txt"), 'w') as f:
        f.write("\n".join(new_labels))

print("\n--- Pré-processamento concluído! ---")

Definir Dataset e Criar DataLoaders

In [ ]:
# Célula 4: Criar Dataset e DataLoaders

# --- ATUALIZANDO VARIÁVEIS DE CAMINHO PARA USAR DADOS PROCESSADOS ---
IMAGE_PATH = '/content/kitti_processed/image_2'
LABEL_PATH = '/content/kitti_processed/label_2'

# --- CLASSE DE DATASET SIMPLIFICADA ---
class KittiDataset(Dataset):
    def __init__(self, image_dir, label_dir, class_mapping, transforms=None):
        self.image_dir = image_dir; self.label_dir = label_dir; self.class_mapping = class_mapping
        self.transforms = transforms
        self.image_ids = sorted([f.split('.')[0] for f in os.listdir(image_dir)])
    def __len__(self): return len(self.image_ids)
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, f"{img_id}.png")
        label_path = os.path.join(self.label_dir, f"{img_id}.txt")
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        boxes, labels = [], []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split(); class_name = parts[0]
                if class_name in self.class_mapping:
                    labels.append(self.class_mapping[class_name])
                    boxes.append(list(map(float, parts[4:8])))
        target = {"boxes": torch.as_tensor(boxes, dtype=torch.float32), "labels": torch.as_tensor(labels, dtype=torch.int64)}
        if self.transforms: image = self.transforms(image)
        return image, target

# --- CRIAÇÃO DATASETS E DATALOADERS ---
CLASSES = ['background', 'Car', 'Pedestrian', 'Cyclist']
CLASS_MAPPING = {'Car': 1, 'Pedestrian': 2, 'Cyclist': 3}
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

full_dataset = KittiDataset(IMAGE_PATH, LABEL_PATH, CLASS_MAPPING, transforms=transforms)

print(f"\nUsando o dataset completo com {len(full_dataset)} imagens.")

# A divisão de 80/20 agora é feita em cima do dataset completo
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

def collate_fn(batch): return tuple(zip(*batch))
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)

print(f"DataLoaders prontos: {len(train_dataset)} treino, {len(val_dataset)} validação.")

Apagar checkpoints


In [ ]:
# Célula para limpar checkpoints antigos
!rm frcnn_kitti_epoch_*.pth
print("Checkpoints antigos foram removidos.")

 Treinamento do FASTER R-CNN com backbone ResNet-50 pré-treinado

In [ ]:
# Célula de Treinamento (com Checkpoints)

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm.notebook import tqdm
import os

# --- DEFINIÇÃO DO MODELO ---
def get_model(num_classes):
    # Carrega um modelo Faster R-CNN com backbone ResNet-50 pré-treinado
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    # Substitui a camada final para o número de classes do dataset
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# Instancia o modelo e o move para a GPU
model_frcnn = get_model(len(CLASSES)).to(device)

# --- OTIMIZADOR E CONFIGURAÇÕES DE TREINO ---
params = [p for p in model_frcnn.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Definição do numero de épocas
num_epochs = 5
start_epoch = 0



checkpoint_to_load = 'frcnn_kitti_epoch_1.pth'

if os.path.exists(checkpoint_to_load):
    print(f"Carregando checkpoint: {checkpoint_to_load}...")
    model_frcnn.load_state_dict(torch.load(checkpoint_to_load, map_location=device))
    try:
        # Extrai o número da época do nome do arquivo para continuar de onde parou
        start_epoch = int(checkpoint_to_load.split('_')[-1].split('.')[0])
        print(f"Checkpoint carregado. O treinamento continuará a partir da época {start_epoch + 1}.")
    except:
        print("Não foi possível extrair o número da época. O treino começará da época 1 com os pesos carregados.")
        start_epoch = 0
else:
    print("Nenhum checkpoint encontrado. Iniciando o treinamento do zero.")


# --- LOOP DE TREINAMENTO ---
print("\n--- INICIANDO TREINAMENTO DO FASTER R-CNN ---")
=
for epoch in range(start_epoch, num_epochs):
    model_frcnn.train() # Coloca o modelo em modo de treino
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}/{num_epochs}")

    for images, targets in progress_bar:
        # Mover dados para a GPU
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Calcular a perda (loss)
        loss_dict = model_frcnn(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # Backpropagation
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
        progress_bar.set_postfix(loss=losses.item())

    print(f"Fim da Época {epoch+1}, Perda Média: {running_loss / len(train_loader):.4f}")

    # --- SALVANDO O CHECKPOINT NO FINAL DE CADA ÉPOCA ---
    checkpoint_path = f'frcnn_kitti_epoch_{epoch+1}.pth'
    torch.save(model_frcnn.state_dict(), checkpoint_path)
    print(f"Checkpoint da Época {epoch+1} salvo em '{checkpoint_path}'")

print("\n--- TREINAMENTO DO FASTER R-CNN CONCLUÍDO ---")

Treinamento para o Faster R-CNN com Backbone MobileNet

In [ ]:
# Célula de Treinamento (com Backbone MobileNet)

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from tqdm.notebook import tqdm
import os

# --- DEFINIÇÃO DO MODELO COM BACKBONE MOBILENET ---
def get_mobilenet_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")


    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


model_mobilenet = get_mobilenet_model(len(CLASSES)).to(device)

# --- OTIMIZADOR E CONFIGURAÇÕES DE TREINO ---
params = [p for p in model_mobilenet.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Treinar por menos épocas, apenas para comparação de velocidade e performance
num_epochs = 5
start_epoch = 0



checkpoint_basename = 'mobilenet_kitti'
checkpoint_to_load = f'{checkpoint_basename}_epoch_1.pth'

if os.path.exists(checkpoint_to_load):
    print(f"Carregando checkpoint: {checkpoint_to_load}...")
    model_mobilenet.load_state_dict(torch.load(checkpoint_to_load, map_location=device))
    try:
        start_epoch = int(checkpoint_to_load.split('_')[-1].split('.')[0])
        print(f"Checkpoint carregado. O treinamento continuará a partir da época {start_epoch + 1}.")
    except:
        print("Não foi possível extrair o número da época. O treino começará da época 1 com os pesos carregados.")
        start_epoch = 0
else:
    print("Nenhum checkpoint do MobileNet encontrado. Iniciando o treinamento do zero.")


# --- LOOP DE TREINAMENTO (USANDO O MODELO MOBILENET) ---
print(f"\\n--- INICIANDO TREINAMENTO DO FASTER R-CNN COM BACKBONE MOBILENET ---")
for epoch in range(start_epoch, num_epochs):
    model_mobilenet.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}/{num_epochs}")

    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model_mobilenet(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
        progress_bar.set_postfix(loss=losses.item())

    print(f"Fim da Época {epoch+1}, Perda Média: {running_loss / len(train_loader):.4f}")

    # --- SALVANDO O CHECKPOINT (COM NOME DIFERENTE) ---
    checkpoint_path = f'{checkpoint_basename}_epoch_{epoch+1}.pth'
    torch.save(model_mobilenet.state_dict(), checkpoint_path)
    print(f"Checkpoint da Época {epoch+1} salvo em '{checkpoint_path}'")

print("\n--- TREINAMENTO DO FASTER R-CNN COM MOBILENET CONCLUÍDO ---")

CÉLULA DE AVALIAÇÃO (APENAS PARA O MODELO COM MOBILENET)

In [ ]:
# CÉLULA DE AVALIAÇÃO (APENAS PARA O MODELO COM MOBILENET)

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm.notebook import tqdm
import numpy as np

# --- 1. CARREGAR O MODELO MOBILENET TREINADO --- # <-- MUDANÇA IMPORTANTE
# Certifique-se de que o nome do arquivo corresponde ao checkpoint final do seu treino com MobileNet
# Se você treinou por 5 épocas, o nome do arquivo será 'mobilenet_kitti_epoch_5.pth'
print("Carregando pesos do modelo com MobileNet...")
model_mobilenet.load_state_dict(torch.load('mobilenet_kitti_epoch_5.pth', map_location=device))


# --- 2. AVALIAÇÃO ---
# A lógica é a mesma, mas usamos a variável 'model_mobilenet'
metric = MeanAveragePrecision(box_format='xyxy')

model_mobilenet.eval() # <-- MUDANÇA
print("\nIniciando avaliação do modelo MobileNet no conjunto de validação...")
progress_bar = tqdm(val_loader, desc="Avaliando MobileNet") # <-- MUDANÇA
with torch.no_grad():
    for images, targets in progress_bar:
        images = list(img.to(device) for img in images)
        predictions = model_mobilenet(images) # <-- MUDANÇA
        formatted_targets = [{"boxes": t["boxes"].to(device), "labels": t["labels"].to(device)} for t in targets]
        metric.update(predictions, formatted_targets)

# 3. Calcular e exibir os resultados finais
print("\n--- Resultados da Avaliação (Faster R-CNN com MobileNet) ---") # <-- MUDANÇA
results = metric.compute()

# Imprimir os resultados gerais
print(f"mAP (IoU=0.50:0.95): {results['map'].item():.4f}")
print(f"mAP (IoU=0.50):      {results['map_50'].item():.4f}")
print(f"mAP (IoU=0.75):      {results['map_75'].item():.4f}")
print("-" * 30)
print(f"Recall Médio:        {results['mar_100'].item():.4f}")
print("-" * 30)

# Impressão de resultados por classe (robusto)
class_ids = np.atleast_1d(results.get('classes', torch.tensor([])).cpu().numpy())
map_per_class = np.atleast_1d(results.get('map_per_class', torch.tensor([])).cpu().numpy())
recall_per_class = np.atleast_1d(results.get('mar_100_per_class', torch.tensor([])).cpu().numpy())
class_map_results = {cid: val for cid, val in zip(class_ids, map_per_class)}
class_recall_results = {cid: val for cid, val in zip(class_ids, recall_per_class)}

for i, class_name in enumerate(CLASSES):
    if i == 0: continue
    class_id = i
    map_val = class_map_results.get(class_id, "N/A")
    recall_val = class_recall_results.get(class_id, "N/A")
    map_str = f"{map_val:.4f}" if isinstance(map_val, (np.floating, float)) else map_val
    recall_str = f"{recall_val:.4f}" if isinstance(recall_val, (np.floating, float)) else recall_val
    print(f"Classe: {class_name} (ID: {class_id})")
    print(f"  - AP @ .50:.95: {map_str}")
    print(f"  - Recall:      {recall_str}")

Avaliação Numérica backbone ResNet-50 (Métricas Oficiais: AP, Recall, Precisão)

In [ ]:
# Célula de Avaliação de Métricas

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm.notebook import tqdm
import numpy as np # Importar numpy

# 1. Inicializar a métrica
metric = MeanAveragePrecision(box_format='xyxy')

# 2. Loop de avaliação (sem alterações)
model_frcnn.eval()
print("Iniciando avaliação no conjunto de validação...")
progress_bar = tqdm(val_loader, desc="Avaliando")
with torch.no_grad():
    for images, targets in progress_bar:
        images = list(img.to(device) for img in images)
        predictions = model_frcnn(images)
        formatted_targets = [{"boxes": t["boxes"].to(device), "labels": t["labels"].to(device)} for t in targets]
        metric.update(predictions, formatted_targets)

# 3. Calcular e exibir os resultados finais
print("\n--- Resultados da Avaliação ---")
results = metric.compute()

# Imprimir os resultados gerais
print(f"mAP (IoU=0.50:0.95): {results['map'].item():.4f}")
print(f"mAP (IoU=0.50):      {results['map_50'].item():.4f}")
print(f"mAP (IoU=0.75):      {results['map_75'].item():.4f}")
print("-" * 30)
print(f"Recall Médio:        {results['mar_100'].item():.4f}")
print("-" * 30)

# --- IMPRESSÃO DE  RESULTADOS POR CLASSE ---
# Usamos np.atleast_1d para garantir que o resultado seja sempre iterável
class_ids = np.atleast_1d(results.get('classes', torch.tensor([])).cpu().numpy())
map_per_class = np.atleast_1d(results.get('map_per_class', torch.tensor([])).cpu().numpy())
recall_per_class = np.atleast_1d(results.get('mar_100_per_class', torch.tensor([])).cpu().numpy())

# Mapeamos o ID da classe ao resultado para busca fácil
class_map_results = {cid: val for cid, val in zip(class_ids, map_per_class)}
class_recall_results = {cid: val for cid, val in zip(class_ids, recall_per_class)}

# Iteração sobre nossas classes conhecidas e buscamos o resultado no dicionário
for i, class_name in enumerate(CLASSES):
    if i == 0: continue # Pular background

    class_id = i
    map_val = class_map_results.get(class_id, "N/A")
    recall_val = class_recall_results.get(class_id, "N/A")

    map_str = f"{map_val:.4f}" if isinstance(map_val, (np.floating, float)) else map_val
    recall_str = f"{recall_val:.4f}" if isinstance(recall_val, (np.floating, float)) else recall_val

    print(f"Classe: {class_name} (ID: {class_id})")
    print(f"  - AP @ .50:.95: {map_str}")
    print(f"  - Recall:      {recall_str}")

Avaliação Visual do Modelo Treinado

In [ ]:
# Célula para Avaliação Visual do Modelo Treinado

# Colocar o modelo em modo de avaliação (importante para resultados consistentes)
model_frcnn.eval()

# Pegar um lote de imagens do CONJUNTO DE VALIDAÇÃO
images, targets = next(iter(val_loader))
images = list(img.to(device) for img in images)

# Fazer as previsões com o seu modelo treinado
with torch.no_grad():
    predictions = model_frcnn(images)

print("Mostrando resultados em algumas imagens de validação...")
print("Caixas VERDES = Previsão do Modelo | Caixas VERMELHAS = Gabarito Real")

# Mover as imagens de volta para a CPU para poder desenhar nelas
images_cpu = [img.cpu() for img in images]

# Loop para visualizar cada imagem e suas detecções
for i in range(len(images_cpu)):
    # Converter o tensor da imagem para um formato que o OpenCV entende
    img_tensor = images_cpu[i]
    img_np = img_tensor.permute(1, 2, 0).numpy()


    # Multiplicamos por 255 para reverter a normalização e ter cores corretas
    img_to_draw = np.ascontiguousarray(img_np * 255).astype(np.uint8)

    # 1. Desenhar as caixas previstas pelo modelo (em VERDE)
    prediction = predictions[i]
    for box, score, label in zip(prediction['boxes'], prediction['scores'], prediction['labels']):
        if score > 0.6: # Limiar de confiança (só desenhar previsões confiáveis)
            box = box.cpu().numpy().astype(int)
            class_name = CLASSES[label.item()]

            # Desenha a caixa
            cv2.rectangle(img_to_draw, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            # Escreve o texto com a classe e a confiança
            cv2.putText(img_to_draw, f"{class_name}: {score:.2f}", (box[0], box[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 2. Desenhar as caixas do gabarito real (em VERMELHO) para comparação
    true_boxes = targets[i]['boxes'].cpu().numpy().astype(int)
    for box in true_boxes:
        cv2.rectangle(img_to_draw, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 1) # Linha mais fina

    # 3. Mostrar a imagem
    plt.figure(figsize=(12, 9))
    plt.imshow(img_to_draw)
    plt.title(f"Resultado na Imagem de Validação {i+1}")
    plt.axis('off')
    plt.show()

ANÁLISE DE CASOS ESPECÍFICOS (ILUMINAÇÃO E OCLUSÃO)

In [ ]:


model_frcnn.eval()


imagens_noturnas_ou_sombra = ['000048', '000077', '000140', '000401', '000222', '000241', '000329']
imagens_com_oclusao = ['000223', '000308', '000338', '000360', '000466', '000000473', '000587'] # Imagens com objetos parcialmente escondidos
imagens_diurnas_boas = ['000335', '000584', '000627', '000678', '000764', '000772', '000787'] # Imagens "fáceis" para comparação


def analisar_imagens(model, image_ids, analysis_title):
    print("\\n" + "="*50)
    print(f"  {analysis_title}")
    print("="*50)
    print("Caixas VERDES = Previsão do Modelo | Caixas VERMELHAS = Gabarito Real")

    for img_id in image_ids:
        img_path = os.path.join(IMAGE_PATH, f"{img_id}.png")
        # Encontrar o target (gabarito) correspondente no val_dataset
        target_original = None
        for img, tgt in val_dataset:

            pass

        image = cv2.imread(img_path)
        image_tensor = torchvision.transforms.ToTensor()(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).to(device)

        with torch.no_grad():
            prediction = model([image_tensor])[0]

        img_to_draw = image.copy()

        # Desenhar previsões (VERDE)
        for box, score, label in zip(prediction['boxes'], prediction['scores'], prediction['labels']):
            if score > 0.6:
                box = box.cpu().numpy().astype(int)
                class_name = CLASSES[label.item()]
                cv2.rectangle(img_to_draw, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
                cv2.putText(img_to_draw, f"{class_name}: {score:.2f}", (box[0], box[1]-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Desenhar gabarito (VERMELHO) - lendo o arquivo de anotação original
        label_path = os.path.join(LABEL_PATH, f"{img_id}.txt")
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if parts[0] in CLASS_MAPPING:
                    box = np.array(list(map(float, parts[4:8]))).astype(int)
                    cv2.rectangle(img_to_draw, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 1)

        plt.figure(figsize=(12, 9))
        plt.imshow(cv2.cvtColor(img_to_draw, cv2.COLOR_BGR2RGB))
        plt.title(f"ID: {img_id}")
        plt.axis('off')
        plt.show()

# --- 4. EXECUTAR A ANÁLISE PARA CADA CATEGORIA ---
analisar_imagens(model_frcnn, imagens_diurnas_boas, "Análise em Condições Diurnas (Boas)")
analisar_imagens(model_frcnn, imagens_noturnas_ou_sombra, "Análise em Condições de Baixa Iluminação / Sombra")
analisar_imagens(model_frcnn, imagens_com_oclusao, "Análise de Desempenho em Oclusão")

Preparação dos Dados para o YOLO

In [ ]:
# PREPARAÇÃO DE DADOS DO YOLO

from ultralytics import YOLO
import shutil
from tqdm.notebook import tqdm
import cv2
import os
import random

# --- 1. DEFINIR CAMINHOS ---
# Caminho dos dados ORIGINAIS (copiados do seu Drive para o ambiente local)
original_images_path = '/content/training_original/image_2' # <-- CORRIGIDO
original_labels_path = '/content/training_original/label_2' # <-- CORRIGIDO

# Caminho de destino para a nova estrutura de pastas do YOLO
yolo_path = '/content/KITTI_YOLO' # Salvaremos em /content/ no Colab
yolo_train_img_dir = os.path.join(yolo_path, 'images', 'train')
yolo_val_img_dir = os.path.join(yolo_path, 'images', 'val')
yolo_train_label_dir = os.path.join(yolo_path, 'labels', 'train')
yolo_val_label_dir = os.path.join(yolo_path, 'labels', 'val')

# Limpar e recriar as pastas de destino
!rm -rf {yolo_path}
os.makedirs(yolo_train_img_dir, exist_ok=True); os.makedirs(yolo_val_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True); os.makedirs(yolo_val_label_dir, exist_ok=True)


# --- 2. CRIAR A DIVISÃO TREINO/VALIDAÇÃO ---
# Pega todos os IDs de imagem, embaralha e divide 80/20
print("Criando divisão de dados para treino e validação...")
all_image_ids = sorted([f.split('.')[0] for f in os.listdir(original_images_path)])
random.seed(42)
random.shuffle(all_image_ids)
split_idx = int(0.8 * len(all_image_ids))
train_image_ids = all_image_ids[:split_idx]
val_image_ids = all_image_ids[split_idx:]
print(f"Divisão criada: {len(train_image_ids)} treino, {len(val_image_ids)} validação.")


# --- 3. FUNÇÃO DE CONVERSÃO ---
def convert_yolo_files(image_ids, subset_name):
    print(f"Convertendo {len(image_ids)} arquivos para o conjunto de {subset_name} do YOLO...")
    img_dest_path = os.path.join(yolo_path, 'images', subset_name)
    label_dest_path = os.path.join(yolo_path, 'labels', subset_name)

    for img_id in tqdm(image_ids, desc=f"Processando {subset_name}"):
        img_name = f"{img_id}.png"
        src_img_path = os.path.join(original_images_path, img_name)
        src_label_path = os.path.join(original_labels_path, f"{img_id}.txt")

        if not os.path.exists(src_img_path) or not os.path.exists(src_label_path):
            continue

        shutil.copy(src_img_path, os.path.join(img_dest_path, img_name))
        img = cv2.imread(os.path.join(img_dest_path, img_name))
        if img is None:
            print(f"Warning: Could not read image file {img_name}. Skipping.")
            continue
        h, w = img.shape[:2]


        with open(src_label_path, 'r') as f_in, open(os.path.join(label_dest_path, f"{img_id}.txt"), 'w') as f_out:
            for line in f_in:
                parts = line.strip().split()
                class_name = parts[0]
                if class_name in CLASS_MAPPING:
                    class_id = CLASS_MAPPING[class_name]
                    # KITTI format is xmin, ymin, xmax, ymax
                    xmin, ymin, xmax, ymax = map(float, parts[4:8])

                    # Convert to YOLO format (x_center, y_center, width, height) normalized
                    x_center = ((xmin + xmax) / 2) / w
                    y_center = ((ymin + ymax) / 2) / h
                    bbox_w = (xmax - xmin) / w
                    bbox_h = (ymax - ymin) / h

                    # YOLO classes are 0-indexed (Car=0, Pedestrian=1, etc.)
                    # Our classes are 1-indexed (Car=1, etc.), so subtract 1.
                    yolo_class_id = class_id - 1

                    # Ensure coordinates are within [0, 1] and format as strings
                    x_center_str = f"{max(0.0, min(1.0, x_center)):.6f}"
                    y_center_str = f"{max(0.0, min(1.0, y_center)):.6f}"
                    bbox_w_str = f"{max(0.0, min(1.0, bbox_w)):.6f}"
                    bbox_h_str = f"{max(0.0, min(1.0, bbox_h)):.6f}"


                    # Write the line in YOLO format: class_id x_center y_center width height
                    f_out.write(f"{yolo_class_id} {x_center_str} {y_center_str} {bbox_w_str} {bbox_h_str}\n")


# --- 4. EXECUTAR A CONVERSION ---
convert_yolo_files(train_image_ids, "train")
convert_yolo_files(val_image_ids, "val")

# --- 5. FINAL VERIFICATION ---
num_train_images = len(os.listdir(yolo_train_img_dir))
print(f"\nVerification: {num_train_images} images were copied to the YOLO train folder.")
if num_train_images == 0:
    raise RuntimeError("The YOLO train folder is empty. The copy script failed.")

# --- 6. CREATE dataset.yaml file ---
yaml_content = f"""
path: {os.path.abspath(yolo_path)}
train: images/train
val: images/val
names:
  0: Car
  1: Pedestrian
  2: Cyclist
"""
yaml_path = os.path.join(yolo_path, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"\nYOLO preparation completed. Configuration file saved to: {yaml_path}")

Treinamento do Modelo YOLO

In [ ]:
# CÉLULA DE TREINAMENTO DO YOLO (AJUSTADA)

from ultralytics import YOLO

# Carregar o modelo YOLOv8 pré-treinado
model_yolo = YOLO('yolov8n.pt')

# Iniciar o treinamento
print("\n--- INICIANDO TREINAMENTO DO YOLOv8 ---")
model_yolo.train(
    data=yaml_path,      # Caminho para o arquivo .yaml que a célula anterior criou
    epochs=20,           # Usar 20 épocas para uma comparação justa com o Faster R-CNN
    imgsz=600,           # <-- MUDANÇA PRINCIPAL: Ajustado para corresponder ao pré-processamento
    project='kitti_results',
    name='yolov8n_comparison'
)

print("\n--- TREINAMENTO DO YOLOv8 CONCLUÍDO ---")

In [ ]:
# Célula para encontrar o caminho correto do treinamento do YOLO
!ls kitti_results/

Avaliação Numérica do YOLO

In [ ]:
# CÉLULA DE AVALIAÇÃO NUMÉRICA DO YOLO

from ultralytics import YOLO

# Carregar o melhor modelo salvo pelo treinamento
best_yolo_model = YOLO('kitti_results/yolov8n_comparison13/weights/best.pt')

# Executar a validação para obter as métricas
print("\n--- Resultados da Avaliação (YOLOv8) ---")
results = best_yolo_model.val()

Análise Visual de Casos Específicos (YOLO)

In [ ]:
# CÉLULA DE ANÁLISE DE CASOS ESPECÍFICOS (YOLO)

import cv2
import matplotlib.pyplot as plt

# --- Suas listas de imagens para análise (USE AS MESMAS DA ANÁLISE DO FASTER R-CNN) ---
# Lembre-se que estas listas contêm apenas os IDs dos arquivos (ex: '000118')
imagens_noturnas_ou_sombra = ['000048', '000077', '000140', '000401', '000222', '000241', '000329']
imagens_com_oclusao = ['000223', '000308', '000338', '000360', '000466', '000000473', '000587'] # Imagens com objetos parcialmente escondidos
imagens_diurnas_boas = ['000335', '000584', '000627', '000678', '000764', '000772', '000787'] # Imagens "fáceis" para comparação

# Carregar o melhor modelo
best_yolo_model = YOLO('kitti_results/yolov8n_comparison13/weights/best.pt')

def analisar_imagens_yolo(model, image_ids, analysis_title):
    print("\n" + "="*50)
    print(f"  {analysis_title} (YOLOv8)")
    print("="*50)

    # Caminho para as imagens originais que foram copiadas para o ambiente
    original_images_path = '/content/training_original/image_2'

    # Filtrar apenas os IDs que realmente existem
    image_paths = []
    for img_id in image_ids:
        path = os.path.join(original_images_path, f"{img_id}.png")
        if os.path.exists(path):
            image_paths.append(path)

    if not image_paths:
        print(f"Nenhuma imagem encontrada para a análise: {analysis_title}")
        return

    # Fazer a predição em todas as imagens de uma vez
    predictions = model.predict(source=image_paths)

    # Mostrar os resultados
    for result in predictions:
        im_array = result.plot() # .plot() já desenha as caixas na imagem
        plt.figure(figsize=(12, 9))
        plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
        plt.title(f"Resultado do YOLOv8 em: {os.path.basename(result.path)}")
        plt.axis('off')
        plt.show()

# Executar a análise visual
analisar_imagens_yolo(best_yolo_model, imagens_diurnas_boas, "Análise em Condições Diurnas (Boas)")
analisar_imagens_yolo(best_yolo_model, imagens_noturnas_ou_sombra, "Análise em Condições de Baixa Iluminação / Sombra")
analisar_imagens_yolo(best_yolo_model, imagens_com_oclusao, "Análise de Desempenho em Oclusão")

In [ ]:
# Célula para Contar a Distribuição das Classes

from collections import Counter

# Dicionário reverso para mapear ID de volta para o nome da classe
inv_class_mapping = {v: k for k, v in CLASS_MAPPING.items()}

# Contadores para cada conjunto de dados
train_counts = Counter()
val_counts = Counter()

print("Analisando o conjunto de treinamento...")
for _, targets in tqdm(train_dataset):
    # .tolist() converte os tensores para uma lista de números
    labels = targets['labels'].tolist()
    train_counts.update(labels)

print("\nAnalisando o conjunto de validação...")
for _, targets in tqdm(val_dataset):
    labels = targets['labels'].tolist()
    val_counts.update(labels)

print("\n--- Distribuição de Classes ---")
print("\nConjunto de Treinamento:")
for label_id, count in sorted(train_counts.items()):
    print(f"  - Classe '{inv_class_mapping[label_id]}': {count} objetos")

print("\nConjunto de Validação:")
for label_id, count in sorted(val_counts.items()):
    print(f"  - Classe '{inv_class_mapping[label_id]}': {count} objetos")

PROPOSTAS DA RPN

In [ ]:
# CÉLULA PARA VISUALIZAR AS PROPOSTAS DA RPN

import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
import numpy as np # Importar numpy

# --- carregamento do melhor checkpoint treinado ---
model_frcnn.load_state_dict(torch.load('frcnn_kitti_epoch_20.pth', map_location=device))
model_frcnn.to(device)
model_frcnn.eval()

# Pegar uma imagem de exemplo do seu val_loader
# Ajuste o índice [0] para visualizar outras imagens
img_tensor, target = val_dataset[0] # Pegar o primeiro item do dataset de validação
img_for_pred = img_tensor.to(device) # Remove unsqueeze(0) here


with torch.no_grad():
    # 1. Obter o mapa de características do backbone
    images = [img_for_pred] # Transform espera uma list a de imagens
    # Ensure images are on the correct device - this is already done above
    images, targets = model_frcnn.transform(images, None)


    features = model_frcnn.backbone(images.tensors)

    # 2. Obter as propostas da RPN
    # A RPN retorna as caixas propostas e suas perdas (que ignoramos na inferência)
    proposals, proposal_losses = model_frcnn.rpn(images, features, None)

# Pegar as caixas propostas para a primeira imagem e mover para a CPU
rpn_boxes = proposals[0].cpu().numpy()

# --- Visualização ---
# Converter a imagem original para um formato que o OpenCV entende
img_cv = img_tensor.permute(1, 2, 0).cpu().numpy().copy() * 255
img_cv = cv2.cvtColor(img_cv.astype('uint8'), cv2.COLOR_RGB2BGR) # Corrigido: cv2.COLOR_RGB2BGR

print(f"A RPN gerou {len(rpn_boxes)} propostas de região para esta imagem.")
print("Desenhando as 100 propostas com maior pontuação...")

# Desenhar as 100 melhores propostas na imagem
# A RPN não dá pontuações de classe, mas sim uma pontuação de "objeto vs não-objeto".
for i, box in enumerate(rpn_boxes[:100]):
    x1, y1, x2, y2 = map(int, box)
    # Desenha um retângulo amarelo semi-transparente
    cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 255), 1)

plt.figure(figsize=(15, 10))
plt.imshow(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
plt.title("Visualização das Propostas da Region Proposal Network (RPN)")
plt.axis('off')
plt.show()